#Inputs: 
Databricks Account ID  
Databricks client ID  
Databricks Client Secret  
Databricks Service Credential for interaction with AWS secrets manager  
Secrets Manager Region  
AWS secrets ARN (optional)  

In [ ]:
%pip install --upgrade databricks-sdk boto3
dbutils.library.restartPython()

In [ ]:
dbutils.widgets.text("secrets_manager_service_credential", "")
dbutils.widgets.text("secrets_manager_region", "")
dbutils.widgets.text("account_id", "")
dbutils.widgets.text("client_id", "")
dbutils.widgets.text("client_secret", "")
dbutils.widgets.text("aws_secret_arn", "")

In [ ]:
import boto3

secrets_manager_service_credential = dbutils.widgets.get("secrets_manager_service_credential")
secrets_manager_region = dbutils.widgets.get("secrets_manager_region")



boto3_session = boto3.Session(botocore_session=dbutils.credentials.getServiceCredentialsProvider(secrets_manager_service_credential ), region_name=secrets_manager_region)
sm = boto3_session.client('secretsmanager')

In [ ]:
from databricks.sdk import AccountClient

a = AccountClient(
host="https://accounts.cloud.databricks.com/",
account_id= dbutils.widgets.get("account_id"),
client_id= dbutils.widgets.get("client_id"),
client_secret= dbutils.widgets.get("client_secret")
)

In [ ]:
import time
sp = a.service_principals.create(
    active = True,
    display_name=f"sdk-{time.time_ns()}"
    )

In [ ]:

sp_oauth_secret = a.service_principal_secrets.create(sp.id)
sp_oauth_secret_dict= sp_oauth_secret.as_dict()

In [ ]:
def generate_secret_name(prefix):
    return f"{prefix}-{time.time_ns()}"
aws_secret_arn = dbutils.widgets.get("aws_secret_arn")

if aws_secret_arn == None or aws_secret_arn == "":
    try: 
        aws_secret = sm.create_secret(Name=generate_secret_name("databricks_waf"), SecretString=json.dumps(sp_oauth_secret_dict))
        aws_secret_arn = aws_secret.get("ARN")
    except Exception as error:
        print(f'could not create the secret in AWS Secrets Manager: {error}')
        raise 
else:
    try: sm.put_secret_value(SecretId=aws_secret_arn, SecretString=json.dumps(secrets_dict))
    except Exception as error:
        print(f'could not update secret {aws_secret_arn} in AWS: {error}')
        raise 






In [ ]:
output = {
    'sp_id': sp.id
    'aws_secret_arn': aws_secret_arn
}